In [36]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
#allows to print the dataframe nicely
from IPython.core import display as ICD
#!pip install plotly
# import plotly.plotly as py
# from plotly.graph_objs import *
# import plotly.tools as tls
# from scipy import stats

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
# import additional packages to insepct data and clean them
import pandas as pd
import os 
import random 
from zipfile import ZipFile
import datetime

In [39]:
# import helping functions from the implementation file
#from proj1_helpers import load_csv_data
from proj1_helpers import *
import implementations
from additional_implementations import *
import seaborn as sns

In [40]:
# import zipped files from the github repository
data_folder='./data/'
zip_file = ZipFile(data_folder+'all.zip')
# zip file creates a list of files with certain properties
zip_file.infolist()

[<ZipInfo filename='train.csv' compress_type=deflate file_size=48103379 compress_size=17670847>,
 <ZipInfo filename='test.csv' compress_type=deflate file_size=109335584 compress_size=40089137>,
 <ZipInfo filename='sample-submission.csv' compress_type=deflate file_size=5682394 compress_size=1263762>]

In [41]:
# now we want to access the 'filename' property in the zipfile variable
# and we create a dictionary of dataframe
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}
df_train=dfs['train.csv']
df_test=dfs['test.csv']
df_sample_submission=dfs['sample-submission.csv']

In [42]:
df_train_pca=df_train.drop(columns=['Prediction', 'Id'])

In [24]:
counts={}
for column_name in df_train_pca.columns.values:
    
    check=df_train_pca[column_name]==-999.0
    counts[column_name]=np.sum(check.astype('int'))
counts

{'DER_mass_MMC': 38114,
 'DER_mass_transverse_met_lep': 0,
 'DER_mass_vis': 0,
 'DER_pt_h': 0,
 'DER_deltaeta_jet_jet': 177457,
 'DER_mass_jet_jet': 177457,
 'DER_prodeta_jet_jet': 177457,
 'DER_deltar_tau_lep': 0,
 'DER_pt_tot': 0,
 'DER_sum_pt': 0,
 'DER_pt_ratio_lep_tau': 0,
 'DER_met_phi_centrality': 0,
 'DER_lep_eta_centrality': 177457,
 'PRI_tau_pt': 0,
 'PRI_tau_eta': 0,
 'PRI_tau_phi': 0,
 'PRI_lep_pt': 0,
 'PRI_lep_eta': 0,
 'PRI_lep_phi': 0,
 'PRI_met': 0,
 'PRI_met_phi': 0,
 'PRI_met_sumet': 0,
 'PRI_jet_num': 0,
 'PRI_jet_leading_pt': 99913,
 'PRI_jet_leading_eta': 99913,
 'PRI_jet_leading_phi': 99913,
 'PRI_jet_subleading_pt': 177457,
 'PRI_jet_subleading_eta': 177457,
 'PRI_jet_subleading_phi': 177457,
 'PRI_jet_all_pt': 0}

In [25]:
#df_train_pca=df_train_pca.drop(columns=['PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'DER_lep_eta_centrality', 'DER_prodeta_jet_jet', 'DER_mass_jet_jet', 'DER_deltaeta_jet_jet','PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi'])

In [26]:
#median=np.median(df_train_pca['DER_mass_MMC'])
#df_train_pca['DER_mass_MMC'][df_train_pca['DER_mass_MMC']==-999.0]=median

In [43]:
df_train_pca[df_train_pca==-999.0]=0

### PCA

In [44]:
features = df_train_pca.columns.values
df_train_pca_std=pd.DataFrame()

df_train_pca_std, mean_train, std_train=standardize_personal(df_train_pca)
df_train_pca_std.head()
print(len(features))

30


In [29]:
cov_df_train_pca_std=np.cov(df_train_pca_std.T)
eig_vals, eig_vecs = np.linalg.eig(cov_df_train_pca_std)

In [30]:
# Make a list of (eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort()
eig_pairs.reverse()

In [ ]:
tot = sum(eig_vals)
var_exp = [(i / tot)*100 for i in sorted(eig_vals, reverse=True)]
cum_var_exp = np.cumsum(var_exp)

trace1 = Bar(
        x=['PC %s' %i for i in range(1,20)],
        y=var_exp,
        showlegend=False)

trace2 = Scatter(
        x=['PC %s' %i for i in range(1,20)], 
        y=cum_var_exp,
        name='cumulative explained variance')

data = Data([trace1, trace2])

layout=Layout(
        yaxis=YAxis(title='Explained variance in percent'),
        title='Explained variance by different principal components')

fig = Figure(data=data, layout=layout)
py.iplot(fig)

In [31]:
number_features=30
#decide how many principal components i get
number_pa=25
#define matrix to be filled in
matrix_w=np.ones((number_features, number_pa))
for i in range(number_pa):
    matrix_w[:,i] = eig_pairs[i][1]

In [32]:
df_train_pca_std_transf = df_train_pca_std.dot(matrix_w)

In [33]:
y_train_pca=np.array(df_train['Prediction'])
y_train_pca[np.where(y_train_pca=='b')] = -1.
y_train_pca[np.where(y_train_pca=='s')] = 1.
y_train_pca=y_train_pca.astype('float')

In [34]:
lambdas=np.logspace(-10,-1,10)
degrees=np.linspace(1,15,15).astype('int')
mses= np.zeros((len(degrees), len(lambdas)))

for ind_degree,degree in enumerate(degrees):
    tx_train_pca_std_transf_polynomial=polynomial_features_simple(df_train_pca_std_transf, degree)
    for ind_lambda,lambda_ in enumerate(lambdas):
        w,loss=implementations.ridge_regression(y_train_pca,tx_train_pca_std_transf_polynomial,lambda_)
        mses[ind_degree, ind_lambda]=loss
        
mse_final, degree_final,lambda_final=get_best_parameters(degrees, lambdas, mses)
tx_train_pca_std_transf_polynomial=polynomial_features_simple(df_train_pca_std_transf, degree_final)
w_final,loss=implementations.ridge_regression(y_train_pca,tx_train_pca_std_transf_polynomial,lambda_final)

print('MSE: ',mse_final,'   Degree: ',degree_final,'   Lambda: ',lambda_final)
print('')
print('Ws: ',w_final)

MSE:  0.31457740951832713    Degree:  13    Lambda:  1e-06

Ws:  [-1.02834031e-01 -8.86880666e-02 -9.62697170e-03  3.64530106e-03
  1.59228366e-03  1.67996988e-04 -2.13359798e-05 -7.11536440e-06
 -7.94164307e-07 -4.92470065e-08 -1.88287832e-09 -4.51301528e-11
 -6.37746490e-13 -4.13175495e-15 -4.99123608e-02 -6.25137703e-02
  1.11283873e-02  5.67196791e-03 -2.18312571e-04 -1.88927524e-04
 -6.19088162e-06  2.32505283e-06  1.91441799e-07 -4.91892221e-09
 -1.13785856e-09 -4.78538244e-11 -6.65447878e-13 -1.79651053e-01
 -1.27129054e-02  1.12061142e-02  2.76451331e-04 -4.93925146e-04
 -9.25950077e-06  8.59668202e-06  1.81697440e-07 -6.11730519e-08
 -1.16024089e-09  1.65299235e-10  1.25051993e-14 -2.57303083e-13
 -2.77810098e-01  4.96642755e-02  2.31347936e-02 -4.25646678e-03
 -6.78673738e-04  1.19856002e-04  7.65967750e-06 -1.46171875e-06
 -2.45673239e-08  7.96729473e-09 -8.34597652e-11 -1.58648744e-11
  4.35589378e-13 -3.94232062e-03 -4.43001293e-02 -1.13612999e-03
  6.65482634e-03 -1.89864

In [35]:
y_pred=predict_labels(w_final,tx_train_pca_std_transf_polynomial)
print('Performance: ',list(y_pred*y_train_pca).count(1.)/len(tx_train_pca_std_transf_polynomial))
print('Degree: ',degree_final)
print('Lambda: ',lambda_final)

Performance:  0.77598
Degree:  13
Lambda:  1e-06


In [ ]:
df_test_pca=df_test.drop(columns=['Prediction', 'Id'])

In [ ]:
df_test_pca=df_test_pca.drop(columns=['PRI_jet_subleading_pt', 'PRI_jet_subleading_eta', 'PRI_jet_subleading_phi', 'DER_lep_eta_centrality', 'DER_prodeta_jet_jet', 'DER_mass_jet_jet', 'DER_deltaeta_jet_jet','PRI_jet_leading_pt','PRI_jet_leading_eta','PRI_jet_leading_phi'])

In [ ]:
df_test_pca['DER_mass_MMC'][df_test_pca['DER_mass_MMC']==-999.0]=median

In [ ]:
features = df_test_pca.columns.values
df_test_pca_std=pd.DataFrame()

# def standardize_test(x, train_set):
#     x=(x-np.mean(train_set, axis=0))/np.std(train_set, axis=0)
#     return x

df_test_pca_std=standardize_test(df_test_pca, mean_train, std_train)
df_test_pca_std.head()

In [ ]:
df_test_pca_std_transf = df_test_pca_std.dot(matrix_w)

In [ ]:
tx_test_pca_std_transf_polynomial=polynomial_features_simple(df_test_pca_std_transf, degree_final)

# Submission

In [ ]:
tx_test=tx_test_pca_std_transf_polynomial
y_pred=predict_labels(w_final,tx_test)
create_csv_submission(df_test['Id'], y_pred, 'trial_ridge_polynomial_pca')